In [1]:
sc

In [2]:
sc.master

'yarn'

In [3]:
global Path
Path = "hdfs://127.0.0.1:9000/user/hughe/data/"

In [4]:
Path

'hdfs://127.0.0.1:9000/user/hughe/data/'

In [5]:
raw_user_data = sc.textFile(Path+"ratings.dat")

In [6]:
raw_user_data.count()

10000054

In [7]:
raw_user_data.first()

'1::122::5::838985046'

In [8]:
from pyspark.mllib.recommendation import Rating

In [9]:
ratings_RDD = raw_user_data.map(lambda line:line.split("::")[:3])

In [10]:
ratings_RDD.take(5)

[['1', '122', '5'],
 ['1', '185', '5'],
 ['1', '231', '5'],
 ['1', '292', '5'],
 ['1', '316', '5']]

In [13]:
users_number = ratings_RDD.map(lambda x:x[0]).distinct().count()

In [15]:
print(users_number)

69878


In [16]:
movies_number = ratings_RDD.map(lambda x:x[1]).distinct().count()

In [17]:
print(movies_number)

10677


In [18]:
from pyspark.mllib.recommendation import ALS

In [20]:
model = ALS.train(ratings_RDD, rank=10, iterations=10, lambda_=0.01)

In [21]:
model

In [22]:
model.recommendProducts(100,5)

[Rating(user=100, product=6556, rating=5.999511209480907),
 Rating(user=100, product=27376, rating=5.893468059138928),
 Rating(user=100, product=6402, rating=5.875665731649503),
 Rating(user=100, product=64285, rating=5.387829038717261),
 Rating(user=100, product=26323, rating=5.37492656127737)]

In [23]:
model.predict(100, 26323)

5.37492656127737

In [24]:
model.recommendUsers(64285, 20)

[Rating(user=29183, product=64285, rating=12.187195770320962),
 Rating(user=12388, product=64285, rating=12.066775771603425),
 Rating(user=14632, product=64285, rating=11.562934960994166),
 Rating(user=57563, product=64285, rating=11.208036612430323),
 Rating(user=12304, product=64285, rating=11.198768318265241),
 Rating(user=27590, product=64285, rating=11.01125457601639),
 Rating(user=56349, product=64285, rating=10.688011635239507),
 Rating(user=31791, product=64285, rating=10.345676803640153),
 Rating(user=66877, product=64285, rating=10.30555195292071),
 Rating(user=7348, product=64285, rating=9.993082706867703),
 Rating(user=40747, product=64285, rating=9.878776814231422),
 Rating(user=18831, product=64285, rating=9.876213931480688),
 Rating(user=3909, product=64285, rating=9.820468828044117),
 Rating(user=63692, product=64285, rating=9.750605734607856),
 Rating(user=12496, product=64285, rating=9.578906037966343),
 Rating(user=69125, product=64285, rating=9.564496886371199),
 Ra

In [25]:
movie_RDD = sc.textFile(Path+"movies.dat")

In [26]:
movie_RDD.first()

'1::Toy Story (1995)::Adventure|Animation|Children|Comedy|Fantasy'

In [27]:
movie_RDD.count()

10681

In [50]:
movie_title = movie_RDD.map(lambda line: line.split("::")).map(lambda x: (int(x[0]),x[1])).collectAsMap()

In [52]:
movie_title

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: '12 Monkeys (Twelve Monkeys) (1995)',
 3

In [35]:
user100 = model.recommendProducts(100,5)

In [40]:
user100[0].rating

5.999511209480907

In [56]:
print("使用者{}，最推薦前五部電影為:".format(user100[0].user))
for i in range(len(user100)):
    print("第{}部:{},推薦指數:{}".format(i+1, movie_title[user100[i].product], round(user100[i].rating, 2)))

使用者100，最推薦前五部電影為:
第1部:Sea Is Watching, The (Umi wa miteita) (2002),推薦指數:6.0
第2部:Tunnel, The (Der Tunnel) (2001),推薦指數:5.89
第3部:Siam Sunset (1999),推薦指數:5.88
第4部:Wallace and Gromit in 'A Matter of Loaf and Death' (2008),推薦指數:5.39
第5部:Groove Tube, The (1974),推薦指數:5.37
